In [ ]:
import matplotlib.pyplot as plt
import mpl_lego as mplego
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
import transformers

from pyprojroot import here
from hate_target import keys
from hate_target import utils

from sklearn.metrics import roc_auc_score, average_precision_score, f1_score
from mpl_lego.labels import bold_text

from hate_measure.nn.classifiers import TargetIdentityClassifier

%matplotlib inline

In [ ]:
path = "~/data/hatecheck/all_cases.csv"

In [ ]:
data = pd.read_csv(path, index_col=0).rename({"target_ident": "target_identity"}, axis=1)

In [ ]:
data['target_race'] = data['target_identity'] == 'black people'
data['target_gender'] = (data['target_identity'] == 'women') | (data['target_identity'] == 'trans people')
data['target_sexuality'] = data['target_identity'] == 'gay people'
data['target_religion'] = data['target_identity'] == 'Muslims'
data['target_disability'] = data['target_identity'] == 'disabled people'
data['target_origin'] = data['target_identity'] == 'immigrants'

In [ ]:
target_groups = sorted(['target_disability', 'target_race', 'target_gender', 'target_religion', 'target_sexuality', 'target_origin'])

In [ ]:
hatecheck = data[['test_case'] + target_groups]

In [ ]:
model_builder = TargetIdentityClassifier.build_model

In [ ]:
model_kwargs = {
    'transformer': 'roberta-large',
    'max_length': 247,
    'n_dense': 256,
    'dropout_rate': 0.05,
    'pooling': 'mean',
    'mask_pool': False
}

In [ ]:
model = model_builder(**model_kwargs)

In [ ]:
model.load_weights('/home/psachdeva/experiments/target_identity/exp07/exp07a_model.h5')

In [ ]:
tokenizer = transformers.RobertaTokenizer.from_pretrained("roberta-large")
tokens = tokenizer(hatecheck['test_case'].tolist(), return_tensors='np', max_length=247, truncation=True, padding='max_length')
inputs = [tokens['input_ids'], tokens['attention_mask']]

In [ ]:
y_pred = model.predict(inputs)

In [ ]:
y_true = [hatecheck[col].values.astype('int') for col in target_groups]
y_pred_subset = [y_pred[i].ravel() for i in [1, 2, 3, 5, 6, 7]]

In [ ]:
accuracies = [(y_true[idx] == (y_pred_subset[idx] >= 0.5).astype('int')).mean() for idx in range(6)]
print(accuracies)

In [ ]:
chance = [1 - y_true[idx].mean() for idx in range(6)]
print(chance)

In [ ]:
f1_scores = [f1_score(y_true[idx], (y_pred_subset[idx] >= 0.5).astype('int')) for idx in range(6)]
print(f1_scores)

In [ ]:
roc_aucs = [roc_auc_score(y_true[idx], y_pred_subset[idx]) for idx in range(6)]
print(roc_aucs)

In [ ]:
pr_aucs = [average_precision_score(y_true[idx], y_pred_subset[idx]) for idx in range(6)]
print(pr_aucs)

In [ ]:
f1_scores = [f1_score(y_true[idx], (y_pred_subset[idx] >= 0.5).astype('int')) for idx in range(6)]
print(f1_scores)